In [ ]:

!pip3 install pynrrd
!pip3 install pyradiomics

#FIGURE OUT WHAT LIBRARIES DO
import SimpleITK as sitk
import itk
import itkwidgets
import matplotlib.pyplot as plt
import math
import numpy as np
import nibabel as nib
import os
import seaborn as sns

import six
import radiomics #used for NGTDM/T1C calculations
from radiomics import featureextractor, getFeatureClasses

from ipywidgets import interact, interactive, IntSlider, ToggleButtons
from numpy import save, load
from scipy.ndimage import zoom
%matplotlib inline
import cv2

#image_path="BraTS-Data/imagesTr/BRATS_001.nii.gz"
#label_path="BraTS-Data/labelsTr/BRATS_001.nii.gz"

#defining sizes of patches + training data folder of modified (x-black border images)
TRAINING_DATA_FOLDER='/Users/user/Desktop/RSNA_ASNR_MICCAI_BraTS2021_TrainingData_16July2021_Subset'
PATCH_SIZE_X = 32
PATCH_SIZE_Y = 32
PATCH_SIZE_Z = 32

#load the image + get the dimensions + values array
def load_image(image_path):
    image_obj=nib.load(image_path)
    print(f'Type of the image {type(image_obj)}')
    image_data=image_obj.get_fdata()
    type(image_data)
    height, width, depth = image_data.shape
    print(f"The image object has the following dimensions: height:{height}, width:{width}, depth:{depth}")
    return image_data

def remove_black_borders(image, label):
    x_nonzero, y_nonzero, z_nonzero = np.nonzero(image)
    cropped_image = image[np.min(x_nonzero):np.max(x_nonzero), np.min(y_nonzero):np.max(y_nonzero), np.min(z_nonzero):np.max(z_nonzero)]
    cropped_label = label[np.min(x_nonzero):np.max(x_nonzero), np.min(y_nonzero):np.max(y_nonzero), np.min(z_nonzero):np.max(z_nonzero)]
    print("Image dimension ")
    print(image.shape)
    print("Cropped image dimension ")
    print(cropped_image.shape)
    return cropped_image, cropped_label
                          
def load_label(label_path):
    label_obj=nib.load(label_path)
    type(label_obj)
    label_data=label_obj.get_fdata()
    type(label_data)
    width, height, depth = label_data.shape
    print(f"Dimensions of labels data array height: {height}, width: {width}, depth: {depth}")
    return label_data

def explore_3dimage(layer):
    plt.figure(figsize=(10, 5))
    plt.imshow(image_data[:, :, layer], cmap='gray')
    plt.title('Explore Layers of Brain MRI', fontsize=20)
    plt.axis('off')
    return layer

def create_label_patches(image_data):
    print("Creating patches")
    patches = []
    patch_locations = []
    width, height, depth = image_data.shape
    nx_patches = width/PATCH_SIZE_X
    ny_patches = height/PATCH_SIZE_Y
    nz_patches = depth/PATCH_SIZE_Z
    center_x = width/2
    center_y = height/2
    center_z = depth/2
    for x in range(0,width,PATCH_SIZE_X):
        for y in range(0,height,PATCH_SIZE_Y):
            for z in range(0,depth,PATCH_SIZE_Z):
                if (((z+PATCH_SIZE_Z) > depth) or ((y+PATCH_SIZE_Y) > height) or ((x+PATCH_SIZE_X) > width)):
                    break
                patch = np.empty((PATCH_SIZE_X,PATCH_SIZE_Y,PATCH_SIZE_Z))
                for px in range(0,PATCH_SIZE_X):
                    for py in range(0,PATCH_SIZE_Y):
                        for pz in range(0,PATCH_SIZE_Z):
                            patch[px][py][pz] = image_data[x+px][y+py][z+pz] 
                patches.append(patch)
                patch_locations.append(z<<18|y<<9|x)
                # print("patch location: %d" % (z<<18|y<<9|x))
                # print("patch count: %d" % len(patches))
            if y+PATCH_SIZE_Y > height:
                break
        if x+PATCH_SIZE_X > width:
            break
    print("Final patch count: %d" % len(patches))
    return patches, patch_locations

def recreate_image_from_patches(patches, patch_locations):
    print("Recreating image from patches")
    image_data = np.empty((512,512,512))
    width, height, depth = image_data.shape
    for patch_index, patch_location in enumerate(patch_locations):
        x = (patch_location & 0x1FF)
        y = (patch_location >> 9) & 0x1FF
        z = (patch_location >> 18) & 0x1FF
        for px in range(0,PATCH_SIZE_X):
            for py in range(0,PATCH_SIZE_Y):
                for pz in range(0,PATCH_SIZE_PZ):
                    image_data[x+px][y+py][z+pz] = patch[patch_index][px][py][pz]                    
    print("Image data: Shape: ")
    print(image_data.shape)
    return image_data

import nrrd

# Initialize feature extractor using the settings file
def create_image_patches(image_data):
    print("Creating patches")
    patches = []
    patch_locations = []
    width, height, depth = image_data.shape
    nx_patches = width/PATCH_SIZE_X
    ny_patches = height/PATCH_SIZE_Y
    nz_patches = depth/PATCH_SIZE_Z
    center_x = width/2
    center_y = height/2
    center_z = depth/2
    for x in range(0,width,PATCH_SIZE_X):
        for y in range(0,height,PATCH_SIZE_Y):
            for z in range(0,depth,PATCH_SIZE_Z):
                if (((z+PATCH_SIZE_Z) > depth) or ((y+PATCH_SIZE_Y) > height) or ((x+PATCH_SIZE_X) > width)):
                    break
                    
                patch = np.empty((2,PATCH_SIZE_X,PATCH_SIZE_Y,PATCH_SIZE_Z))
                for px in range(0,PATCH_SIZE_X):
                    for py in range(0,PATCH_SIZE_Y):
                        for pz in range(0,PATCH_SIZE_Z):
                            patch[0][px][py][pz] = image_data[x+px][y+py][z+pz]
                            
                for px in range(0,PATCH_SIZE_X):
                    for py in range(0,PATCH_SIZE_Y):
                        for pz in range(0,PATCH_SIZE_Z):
                            minimum = 99999
                            for mx in [-1,0,1]:
                                for my in [-1,0,1]:
                                    for mz in [-1,0,1]:
                                        if ((px+mx >=0) and (px+mx < PATCH_SIZE_X)
                                            and (py+my >=0) and (py+my < PATCH_SIZE_Y)
                                            and (pz+mz >=0) and (pz+mz < PATCH_SIZE_Z)):
                                            if minimum > patch[0][px+mx][py+my][pz+mz]:
                                                minimum = patch[0][px+mx][py+my][pz+mz]
                            patch[1][px][py][pz] = minimum
                
                ''' TBU 
                # Use pyradiomics to get NGTM
                imageName = 'testdata.nrrd'
                nrrd.write(imageName, np.array(patch[0]))
                maskName = "testdata_mask.nrrd"
                mask_data = np.ones((PATCH_SIZE_X,PATCH_SIZE_Y,PATCH_SIZE_Z))
                mask_data[PATCH_SIZE_X-2][PATCH_SIZE_Y-2][PATCH_SIZE_Z-2] = 0
                nrrd.write(maskName, mask_data)

                # print("Starting to extracting features")
                featureVector = extractor.execute(imageName, maskName, voxelBased=True)
                # print("Completed extracting features")
                
                ngtdm_data = np.empty((width, height, depth))
                for featureName, featureValue in six.iteritems(featureVector):
                    # if featureName == "original_ngtdm_Coarseness":
                    if featureName == "original_ngtdm_Strength":
                        # print('%s: %s' % (featureName, featureValue))
                        if isinstance(featureValue, sitk.Image):
                            ngtdm_file = featureName + '.nrrd'
                            sitk.WriteImage(featureValue, ngtdm_file)
                            # print('Computed %s, stored as %s' % (featureName, ngtdm_file))
                            ngtdm_data, header = nrrd.read(ngtdm_file)
                             
                for px in range(0,PATCH_SIZE_X):
                    for py in range(0,PATCH_SIZE_Y):
                        for pz in range(0,PATCH_SIZE_Z):
                            patch[2][px][py][pz] = ngtdm_data[px][py][pz]
                            distance = math.sqrt((x+px-center_x)**2 + (y+py-center_y)**2 + (z+pz-center_z)**2) 
                            # patch[3][px][py][pz] = distance
                TBU '''
                
                patches.append(patch)
                patch_locations.append(z<<18|y<<9|x)
                # print("patch location: %d" % (z<<18|y<<9|x))
                # print("patch count: %d" % len(patches))
            if y+PATCH_SIZE_Y > height:
                break
        if x+PATCH_SIZE_X > width:
            break
    print("Final patch count: %d" % len(patches))
    return patches, patch_locations

def save_image_patches(patient_id, patches, patch_locations):
    print("Saving patches for patient %s", patient_id)
    for patch_num, patch in enumerate(patches):
        patch_array = np.array(patch)
        output_file = 'NIFTI-2/nifti-' + patient_id + '-' + str(patch_locations[patch_num]) + '.npy'
        save(output_file, patch_array)
    return patches

def save_label_patches(patient_id, patches, patch_locations):
    print("Saving patches for patient %s num patches: %d", patient_id, len(patches))
    for patch_num, patch in enumerate(patches):
        output_file = 'NIFTI-2/label-' + patient_id + '-' + str(patch_locations[patch_num]) + '.npy'
        save(output_file, patch)
    return patches

def load_image_patch(patient_id, patch_num):
    print("Loading patch, Patient id: %s, Patch num: %d" % (patient_id, patch_num))
    patch_file = 'NIFTI-2/nifti-' + patient_id + '-' + str(patch_num) + '.npy'
    patch_array = np.load(patch_file)
    print(patch_array)

def load_label_patch(patient_id, patch_num):
    print("Loading patch, Patient id: %s" % (patient_id))
    patch_file = 'NIFTI-2/label-' + patient_id + '-' + str(patch_num)
    patch_array = np.load(patch_file)
    for x in range(0,PATCH_SIZE_X):
        for y in range(0,PATCH_SIZE_Y):
            for z in range(0,PATCH_SIZE_Z):
                if (patch_array[x][y][z] != 0):
                    print("2:Found non-zero label: %d", patch_array[x][y][z])
    print(patch_array)

# Read the data back from file
# readdata, header = nrrd.read(filename)

# parameters for pyRadiomics pack (image type, FLAIR NGTDM coarseness, standard settings)
paramsFile = os.path.abspath(r'testdata.yaml')
num_patients = 0
# calling the featureextractor from pyRadiomics package
extractor = featureextractor.RadiomicsFeatureExtractor(paramsFile)
for patient_folder in os.listdir(TRAINING_DATA_FOLDER):
    patient_directory = TRAINING_DATA_FOLDER + "/" + patient_folder
    num_patients=num_patients+1
    print('num_patients: %d' % num_patients)
    if os.path.isdir(patient_directory):    
        patient_id = patient_folder.split('_')[1]
        print("patient %s" % patient_id)
        image_path = TRAINING_DATA_FOLDER + "/" + patient_folder + "/" + patient_folder + "_t1ce.nii.gz"
        image_data = load_image(image_path)
        label_path = TRAINING_DATA_FOLDER + "/" + patient_folder + "/" + patient_folder + "_seg.nii.gz"
        label_data = load_label(label_path)
        image_data, label_data = remove_black_borders(image_data, label_data)
        imgage_data = cv2.normalize(image_data, None, 0, 255, cv2.NORM_MINMAX)
        patches, patch_locations = create_image_patches(image_data)
        print("number of patches %d" % (len(patches)))
        save_image_patches(patient_id, patches, patch_locations)
        patches, patch_locations = create_label_patches(label_data)
        save_label_patches(patient_id, patches, patch_locations)

# interact(explore_3dimage, layer=(0, image_data.shape[2]-1))

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
You should consider upgrading via the '/usr/local/opt/python@3.9/bin/python3.9 -m pip install --upgrade pip' command.
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no lon

Image dimension 
(240, 240, 155)
Cropped image dimension 
(134, 169, 128)
Creating patches
Final patch count: 80
number of patches 80
Saving patches for patient %s 00035
Creating patches
Final patch count: 80
Saving patches for patient %s num patches: %d 00035 80
num_patients: 12
patient 00207
Type of the image <class 'nibabel.nifti1.Nifti1Image'>
The image object has the following dimensions: height:240, width:240, depth:155
Dimensions of labels data array height: 240, width: 240, depth: 155
Image dimension 
(240, 240, 155)
Cropped image dimension 
(131, 169, 142)
Creating patches
Final patch count: 80
number of patches 80
Saving patches for patient %s 00207
Creating patches
Final patch count: 80
Saving patches for patient %s num patches: %d 00207 80
num_patients: 13
patient 00238
Type of the image <class 'nibabel.nifti1.Nifti1Image'>
The image object has the following dimensions: height:240, width:240, depth:155
Dimensions of labels data array height: 240, width: 240, depth: 155
Imag

The image object has the following dimensions: height:240, width:240, depth:155
Dimensions of labels data array height: 240, width: 240, depth: 155
Image dimension 
(240, 240, 155)
Cropped image dimension 
(132, 173, 134)
Creating patches
Final patch count: 80
number of patches 80
Saving patches for patient %s 00002
Creating patches
Final patch count: 80
Saving patches for patient %s num patches: %d 00002 80
num_patients: 29
patient 00206
Type of the image <class 'nibabel.nifti1.Nifti1Image'>
The image object has the following dimensions: height:240, width:240, depth:155
Dimensions of labels data array height: 240, width: 240, depth: 155
Image dimension 
(240, 240, 155)
Cropped image dimension 
(136, 176, 139)
Creating patches
Final patch count: 80
number of patches 80
Saving patches for patient %s 00206
Creating patches
Final patch count: 80
Saving patches for patient %s num patches: %d 00206 80
num_patients: 30
patient 00239
Type of the image <class 'nibabel.nifti1.Nifti1Image'>
The 

Final patch count: 80
number of patches 80
Saving patches for patient %s 00176
Creating patches
Final patch count: 80
Saving patches for patient %s num patches: %d 00176 80
num_patients: 46
patient 00171
Type of the image <class 'nibabel.nifti1.Nifti1Image'>
The image object has the following dimensions: height:240, width:240, depth:155
Dimensions of labels data array height: 240, width: 240, depth: 155
Image dimension 
(240, 240, 155)
Cropped image dimension 
(140, 176, 135)
Creating patches
Final patch count: 80
number of patches 80
Saving patches for patient %s 00171
Creating patches
Final patch count: 80
Saving patches for patient %s num patches: %d 00171 80
num_patients: 47
patient 00185
Type of the image <class 'nibabel.nifti1.Nifti1Image'>
The image object has the following dimensions: height:240, width:240, depth:155
Dimensions of labels data array height: 240, width: 240, depth: 155
Image dimension 
(240, 240, 155)
Cropped image dimension 
(140, 167, 140)
Creating patches
Fina

The image object has the following dimensions: height:240, width:240, depth:155
Dimensions of labels data array height: 240, width: 240, depth: 155
Image dimension 
(240, 240, 155)
Cropped image dimension 
(135, 178, 139)
Creating patches
Final patch count: 80
number of patches 80
Saving patches for patient %s 00191
Creating patches
Final patch count: 80
Saving patches for patient %s num patches: %d 00191 80
num_patients: 63
patient 00154
Type of the image <class 'nibabel.nifti1.Nifti1Image'>
The image object has the following dimensions: height:240, width:240, depth:155
Dimensions of labels data array height: 240, width: 240, depth: 155
Image dimension 
(240, 240, 155)
Cropped image dimension 
(142, 164, 140)
Creating patches
Final patch count: 80
number of patches 80
Saving patches for patient %s 00154
Creating patches
Final patch count: 80
Saving patches for patient %s num patches: %d 00154 80
num_patients: 64
patient 00108
Type of the image <class 'nibabel.nifti1.Nifti1Image'>
The 

Final patch count: 80
number of patches 80
Saving patches for patient %s 00087
Creating patches
Final patch count: 80
Saving patches for patient %s num patches: %d 00087 80
num_patients: 79
patient 00241
Type of the image <class 'nibabel.nifti1.Nifti1Image'>
The image object has the following dimensions: height:240, width:240, depth:155
Dimensions of labels data array height: 240, width: 240, depth: 155
Image dimension 
(240, 240, 155)
Cropped image dimension 
(137, 171, 139)
Creating patches
Final patch count: 80
number of patches 80
Saving patches for patient %s 00241
Creating patches
Final patch count: 80
Saving patches for patient %s num patches: %d 00241 80
num_patients: 80
patient 00246
Type of the image <class 'nibabel.nifti1.Nifti1Image'>
The image object has the following dimensions: height:240, width:240, depth:155
Dimensions of labels data array height: 240, width: 240, depth: 155
Image dimension 
(240, 240, 155)
Cropped image dimension 
(132, 178, 137)
Creating patches
Fina